In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
 
# Machine Learning Models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# Model Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from sklearn.metrics import classification_report, accuracy_score
import random
random.seed(100)


In [2]:
# Specify the path to your CSV file
file_path = 'D:/Naira Documents/Git_projects/MediClaim_Fraud_Detection/Data/training_data.csv'

# Load the CSV data into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())

   Unnamed: 0     BeneID   ClaimID  Provider  InscClaimAmtReimbursed  \
0           0  BENE11001  CLM46614  PRV55912                   26000   
1           1  BENE11001  CLM66048  PRV55907                    5000   
2           2  BENE11001  CLM68358  PRV56046                    5000   
3           3  BENE11011  CLM38412  PRV52405                    5000   
4           4  BENE11014  CLM63689  PRV56614                   10000   

   DeductibleAmtPaid  ClaimPeriod  TimeInHptal  Diagnosis Count  \
0             1068.0            6            6              9.0   
1             1068.0            2            2              3.0   
2             1068.0            3            3              6.0   
3             1068.0            8            8              9.0   
4             1068.0           17           17              9.0   

   Procedures Count  SamePhysician  OPD_Flag  PotentialFraud         DOB  \
0               0.0            0.0         0               1  1943-01-01   
1           

In [3]:
df.isna().sum()


Unnamed: 0                         0
BeneID                             0
ClaimID                            0
Provider                           0
InscClaimAmtReimbursed             0
DeductibleAmtPaid                  0
ClaimPeriod                        0
TimeInHptal                        0
Diagnosis Count                    0
Procedures Count                   0
SamePhysician                      0
OPD_Flag                           0
PotentialFraud                     0
DOB                                0
DOD                                0
Gender                             0
Race                               0
RenalDiseaseIndicator              0
State                              0
County                             0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
C

In [4]:
del df['Unnamed: 0']

In [5]:
df.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,PotentialFraud,RenalDiseaseIndicator,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.00000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.0,558211.000000
mean,997.012133,78.293178,1.727940,0.410762,3.010897,0.053557,0.112248,0.927493,0.381211,0.196786,378.588195,11.931472,11.93877,5227.971466,568.756807,2278.225348,649.698745,1935.723180,73.769770,0.0,4.498616
std,3821.534891,273.814592,4.904984,2.112693,2.448213,0.280534,0.315671,0.259325,0.485685,0.397569,265.215531,0.889712,0.78590,11786.274732,1179.172616,3881.846386,1002.020811,13.011761,13.022524,0.0,2.332301
min,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-8000.000000,0.000000,-70.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.00000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,350.000000,12.000000,12.00000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,1.000000,0.000000,570.000000,12.000000,12.00000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,1.000000,1.000000,1.000000,999.000000,12.000000,12.00000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [6]:
df[df['DeductibleAmtPaid']== -1].head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,PotentialFraud,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
20,BENE11057,CLM36789,PRV51393,13000,-1.0,35,35,9.0,1.0,0.0,0,1,1933-02-01,2009-12-01,Female,White,0,California,570,12,12,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,No,41080,2136,200,50,1933,77.0,0,6.0
21,BENE11057,CLM38115,PRV51342,10000,-1.0,6,6,9.0,0.0,0.0,0,0,1933-02-01,2009-12-01,Female,White,0,California,570,12,12,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,No,41080,2136,200,50,1933,77.0,0,6.0
129,BENE11494,CLM66768,PRV51501,18000,-1.0,10,10,9.0,1.0,0.0,0,1,1931-01-01,2009-12-01,Female,White,0,California,200,12,12,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,No,Yes,No,29500,3204,3510,60,1931,79.0,0,8.0
159,BENE11592,CLM34790,PRV55262,6000,-1.0,5,5,9.0,0.0,0.0,0,0,1931-04-01,2009-12-01,Female,White,0,North Carolina,440,12,12,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,No,60000,10204,310,30,1931,79.0,0,8.0
177,BENE11670,CLM37086,PRV52019,400,-1.0,3,3,7.0,0.0,0.0,0,1,1920-11-01,2009-12-01,Female,White,0,Florida,590,12,12,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,No,76460,4272,3110,470,1920,89.0,0,7.0


In [7]:
df['DeductibleAmtPaid'] = df['DeductibleAmtPaid'].replace(-1,0)

In [8]:
df.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,PotentialFraud,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
0,BENE11001,CLM46614,PRV55912,26000,1068.0,6,6,9.0,0.0,0.0,0,1,1943-01-01,2009-12-01,Male,White,0,Pennsylvania,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
1,BENE11001,CLM66048,PRV55907,5000,1068.0,2,2,3.0,1.0,1.0,0,0,1943-01-01,2009-12-01,Male,White,0,Pennsylvania,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
2,BENE11001,CLM68358,PRV56046,5000,1068.0,3,3,6.0,0.0,0.0,0,0,1943-01-01,2009-12-01,Male,White,0,Pennsylvania,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
3,BENE11011,CLM38412,PRV52405,5000,1068.0,8,8,9.0,1.0,0.0,0,0,1914-03-01,2009-12-01,Female,Black or African American,0,Alabama,360,12,12,No,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes,5000,1068,250,320,1914,96.0,0,6.0
4,BENE11014,CLM63689,PRV56614,10000,1068.0,17,17,9.0,1.0,0.0,0,0,1938-04-01,2009-12-01,Female,White,1,Utah,780,12,12,No,Yes,Yes,No,Yes,Yes,No,Yes,No,No,No,21260,2136,120,100,1938,72.0,0,5.0


In [9]:
df.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,PotentialFraud,RenalDiseaseIndicator,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.00000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.0,558211.000000
mean,997.012133,78.294788,1.727940,0.410762,3.010897,0.053557,0.112248,0.927493,0.381211,0.196786,378.588195,11.931472,11.93877,5227.971466,568.756807,2278.225348,649.698745,1935.723180,73.769770,0.0,4.498616
std,3821.534891,273.814128,4.904984,2.112693,2.448213,0.280534,0.315671,0.259325,0.485685,0.397569,265.215531,0.889712,0.78590,11786.274732,1179.172616,3881.846386,1002.020811,13.011761,13.022524,0.0,2.332301
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-8000.000000,0.000000,-70.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.00000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,350.000000,12.000000,12.00000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,1.000000,0.000000,570.000000,12.000000,12.00000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,1.000000,1.000000,1.000000,999.000000,12.000000,12.00000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [10]:
df['OPD_Flag'] = df['OPD_Flag'].astype('object')
df['SamePhysician'] = df['SamePhysician'].astype('object')
df['RenalDiseaseIndicator'] = df['RenalDiseaseIndicator'].astype('object')

In [11]:
df.describe()


,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.00000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.000000,558211.0,558211.000000
mean,997.012133,78.294788,1.727940,0.410762,3.010897,0.053557,0.381211,378.588195,11.931472,11.93877,5227.971466,568.756807,2278.225348,649.698745,1935.723180,73.769770,0.0,4.498616
std,3821.534891,273.814128,4.904984,2.112693,2.448213,0.280534,0.485685,265.215531,0.889712,0.78590,11786.274732,1179.172616,3881.846386,1002.020811,13.011761,13.022524,0.0,2.332301
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-8000.000000,0.000000,-70.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.00000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.00000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.00000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.00000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [12]:
df[df['IPAnnualReimbursementAmt']<0].shape

(57, 40)

In [13]:
df[df['OPAnnualReimbursementAmt']<0].shape

(16, 40)

In [14]:
df[(df['IPAnnualReimbursementAmt']<0) & (df['OPAnnualReimbursementAmt']<0)].shape

(0, 40)

In [15]:
df = df[df['IPAnnualReimbursementAmt']>=0].reset_index(drop=True)
df = df[df['OPAnnualReimbursementAmt']>=0].reset_index(drop=True)
df.shape

(558138, 40)

In [16]:
df.tail()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,PotentialFraud,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
558133,BENE159198,CLM510792,PRV53699,800,0.0,0,0,3.0,0.0,1.0,1,0,1952-04-01,2009-12-01,Female,White,0,Maryland,20,12,12,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No,0,0,5470,1870,1952,58.0,0,5.0
558134,BENE159198,CLM551294,PRV53702,400,0.0,0,0,3.0,0.0,0.0,1,0,1952-04-01,2009-12-01,Female,White,0,Maryland,20,12,12,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No,0,0,5470,1870,1952,58.0,0,5.0
558135,BENE159198,CLM596444,PRV53676,60,0.0,0,0,2.0,0.0,0.0,1,0,1952-04-01,2009-12-01,Female,White,0,Maryland,20,12,12,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No,0,0,5470,1870,1952,58.0,0,5.0
558136,BENE159198,CLM636992,PRV53689,70,0.0,0,0,0.0,0.0,0.0,1,0,1952-04-01,2009-12-01,Female,White,0,Maryland,20,12,12,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No,0,0,5470,1870,1952,58.0,0,5.0
558137,BENE159198,CLM686139,PRV53689,80,0.0,1,0,8.0,0.0,0.0,1,0,1952-04-01,2009-12-01,Female,White,0,Maryland,20,12,12,Yes,Yes,No,No,No,Yes,Yes,No,No,Yes,No,0,0,5470,1870,1952,58.0,0,5.0


In [17]:
df.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.0,558138.000000
mean,996.806184,78.275842,1.727711,0.410630,3.010793,0.053537,0.381212,378.587666,11.931463,11.938762,5228.095847,568.674654,2278.394824,649.732926,1935.723183,73.769772,0.0,4.498626
std,3820.614730,273.782870,4.904651,2.112183,2.448085,0.280484,0.485685,265.210830,0.889769,0.785951,11785.791749,1179.216729,3882.045712,1002.070824,13.011398,13.022134,0.0,2.332332
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.000000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.000000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.000000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.000000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [18]:
df[df['InscClaimAmtReimbursed']>100000].head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,PotentialFraud,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
10,BENE11034,CLM57949,PRV55193,102000,1068.0,8,8,9.0,1.0,0.0,0,1,1946-03-01,2009-12-01,Female,White,0,North Carolina,760,12,12,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes,131140,2136,1650,80,1946,64.0,0,9.0
708,BENE13669,CLM67392,PRV55172,125000,1068.0,35,35,7.0,2.0,0.0,0,1,1941-05-01,2009-12-01,Male,Black or African American,0,North Carolina,730,12,12,No,Yes,No,No,No,No,No,Yes,No,No,No,139400,1068,102960,956,1941,69.0,0,2.0
832,BENE14036,CLM54264,PRV54942,124000,0.0,21,21,9.0,1.0,0.0,0,1,1917-07-01,2009-12-01,Female,Black or African American,0,New York,770,12,12,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,No,148580,2136,4200,810,1917,92.0,0,7.0
3075,BENE22137,CLM59789,PRV52077,112000,1068.0,12,12,8.0,0.0,0.0,0,1,1927-01-01,2009-12-01,Male,White,1,Florida,510,12,12,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,153580,5640,5890,2070,1927,83.0,0,9.0
3998,BENE25318,CLM34029,PRV51399,103000,1068.0,7,7,10.0,1.0,0.0,0,1,1938-09-01,2009-12-01,Male,White,0,California,370,12,12,No,Yes,No,No,No,No,No,Yes,No,No,Yes,103490,1068,85160,1476,1938,71.0,0,3.0


### Columns to treat that contains outliers
1. InscClaimAmtReimbursed
2. IPAnnualReimbursement
3. OPAnnualReimbursement
4. IPAnnualDeductibleAmt
5. OPAnnualDeductibleAmt

### Outlier Treatment

In [19]:
df.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.0,558138.000000
mean,996.806184,78.275842,1.727711,0.410630,3.010793,0.053537,0.381212,378.587666,11.931463,11.938762,5228.095847,568.674654,2278.394824,649.732926,1935.723183,73.769772,0.0,4.498626
std,3820.614730,273.782870,4.904651,2.112183,2.448085,0.280484,0.485685,265.210830,0.889769,0.785951,11785.791749,1179.216729,3882.045712,1002.070824,13.011398,13.022134,0.0,2.332332
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.000000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.000000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.000000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,125000.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.000000,161470.000000,38272.000000,102960.000000,13840.000000,1983.000000,101.000000,0.0,11.000000


In [20]:
df[df['DeductibleAmtPaid']>0].shape

(60592, 40)

In [22]:
def treat_outliers(df, columns):
   
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR\
        # Cap outliers
        df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
        df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df

In [23]:
df = treat_outliers(df, ['InscClaimAmtReimbursed', 'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt'])

In [24]:
df.describe()

,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
count,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.000000,558138.0,558138.000000
mean,214.277258,78.275842,1.727711,0.410630,3.010793,0.053537,0.381212,378.587666,11.931463,11.938762,3346.451523,502.191917,1775.557649,533.814659,1935.723183,73.769772,0.0,4.498626
std,248.572190,273.782870,4.904651,2.112183,2.448085,0.280484,0.485685,265.210830,0.889769,0.785951,5434.170231,785.740737,1692.759775,526.272005,13.011398,13.022134,0.0,2.332332
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1909.000000,26.000000,0.0,0.000000
25%,40.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,150.000000,12.000000,12.000000,0.000000,0.000000,460.000000,120.000000,1927.000000,68.000000,0.0,3.000000
50%,80.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,350.000000,12.000000,12.000000,0.000000,0.000000,1170.000000,340.000000,1935.000000,75.000000,0.0,5.000000
75%,300.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,570.000000,12.000000,12.000000,6000.000000,1068.000000,2590.000000,790.000000,1941.000000,82.000000,0.0,6.000000
max,690.000000,1068.000000,36.000000,35.000000,10.000000,5.000000,1.000000,999.000000,12.000000,12.000000,15000.000000,2670.000000,5785.000000,1795.000000,1983.000000,101.000000,0.0,11.000000


In [25]:
df.drop(columns={'DOB', 'DOD', 'Alive', 'State', 'County'}, inplace=True)

In [27]:
df.columns

Index(['BeneID', 'ClaimID', 'Provider', 'InscClaimAmtReimbursed',
       'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag', 'PotentialFraud',
       'Gender', 'Race', 'RenalDiseaseIndicator', 'NoOfMonths_PartACov',
       'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
       'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
       'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'BirthYear', 'Age',
       'ChronicDisease_Count'],
      dtype='object')

### Categorical Columns Overview

In [28]:
df.select_dtypes(include='object').columns

Index(['BeneID', 'ClaimID', 'Provider', 'SamePhysician', 'OPD_Flag', 'Gender',
       'Race', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke'],
      dtype='object')

In [30]:
df['Race'].value_counts()

Race
White                        470982
Black or African American     55628
Other                         19708
Hispanic                      11820
Name: count, dtype: int64

In [38]:
df['RenalDiseaseIndicator'] = df['RenalDiseaseIndicator'].replace({0: 'No', 1: 'Yes'})
df['OPD_Flag'] = df['OPD_Flag'].replace({0: 'No', 1: 'Yes'})
df['SamePhysician'] = df['SamePhysician'].astype(int).replace({0: 'No', 1: 'Yes'})


### Base Model

In [33]:
df['ClaimID'].nunique()

558138

In [34]:
del df['BeneID']

In [35]:
df.shape

(558138, 34)

#### One-hot encoding

In [39]:
pd.get_dummies(df, columns= cat_cols, drop_first=True)


,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,PotentialFraud,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,ChronicDisease_Count,SamePhysician_Yes,OPD_Flag_Yes,Gender_Male,Race_Hispanic,Race_Other,Race_White,RenalDiseaseIndicator_Yes,ChronicCond_Alzheimer_Yes,ChronicCond_Heartfailure_Yes,ChronicCond_KidneyDisease_Yes,ChronicCond_Cancer_Yes,ChronicCond_ObstrPulmonary_Yes,ChronicCond_Depression_Yes,ChronicCond_Diabetes_Yes,ChronicCond_IschemicHeart_Yes,ChronicCond_Osteoporasis_Yes,ChronicCond_rheumatoidarthritis_Yes,ChronicCond_stroke_Yes
0,CLM46614,PRV55912,690.0,1068.0,6,6,9.0,0.0,1,12,12,15000.0,2670.0,60.0,70.0,1943,67.0,7.0,False,False,True,False,False,True,False,True,False,True,False,False,True,True,True,False,True,True
1,CLM66048,PRV55907,690.0,1068.0,2,2,3.0,1.0,0,12,12,15000.0,2670.0,60.0,70.0,1943,67.0,7.0,True,False,True,False,False,True,False,True,False,True,False,False,True,True,True,False,True,True
2,CLM68358,PRV56046,690.0,1068.0,3,3,6.0,0.0,0,12,12,15000.0,2670.0,60.0,70.0,1943,67.0,7.0,False,False,True,False,False,True,False,True,False,True,False,False,True,True,True,False,True,True
3,CLM38412,PRV52405,690.0,1068.0,8,8,9.0,1.0,0,12,12,5000.0,1068.0,250.0,320.0,1914,96.0,6.0,False,False,False,False,False,False,False,False,True,True,False,False,True,True,False,False,True,True
4,CLM63689,PRV56614,690.0,1068.0,17,17,9.0,1.0,0,12,12,15000.0,2136.0,120.0,100.0,1938,72.0,5.0,False,False,False,False,False,True,True,False,True,True,False,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558133,CLM510792,PRV53699,690.0,0.0,0,0,3.0,0.0,0,12,12,0.0,0.0,5470.0,1795.0,1952,58.0,5.0,True,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False
558134,CLM551294,PRV53702,400.0,0.0,0,0,3.0,0.0,0,12,12,0.0,0.0,5470.0,1795.0,1952,58.0,5.0,False,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False
558135,CLM596444,PRV53676,60.0,0.0,0,0,2.0,0.0,0,12,12,0.0,0.0,5470.0,1795.0,1952,58.0,5.0,False,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False
558136,CLM636992,PRV53689,70.0,0.0,0,0,0.0,0.0,0,12,12,0.0,0.0,5470.0,1795.0,1952,58.0,5.0,False,True,False,False,False,True,False,True,True,False,False,False,True,True,False,False,True,False


In [42]:
cat_cols = ['SamePhysician', 'OPD_Flag', 'Gender',
       'Race', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']

data_encoded = pd.get_dummies(df, columns= cat_cols, drop_first=True)
data_encoded.shape

(558138, 36)

In [43]:
X = data_encoded.drop(columns='PotentialFraud')
y = data_encoded['PotentialFraud']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=42, stratify=y)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (390696, 35)
X_test: (167442, 35)
y_train: (390696,)
y_test: (167442,)


In [46]:
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

(PotentialFraud
 0    0.618788
 1    0.381212
 Name: proportion, dtype: float64,
 PotentialFraud
 0    0.618787
 1    0.381213
 Name: proportion, dtype: float64)

In [47]:
lr_model = LogisticRegression(class_weight='balanced', random_state=42)
lr_model.fit(X_train.drop(['ClaimID', 'Provider'], axis=1), y_train)

d:\Naira Documents\Git_projects\MediClaim_Fraud_Detection\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [48]:
import mlflow
import mlflow.sklearn
 
try:
    # Set MLflow Tracking URI using SQL Server and Windows Authentication
    mlflow.set_tracking_uri("mssql+pyodbc://DESKTOP-KI17L18\SQLEXPRESS/Medicare_claim_SMA?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")
 
    # Name the experiment
    mlflow.set_experiment("MedicareClaim_Fraud_Detection")
except Exception as e:
    print(f"Connection failed: {e}")

ModuleNotFoundError: No module named 'mlflow'

Will Consider Recall because we need to reduce false Positive cases

###